In [1]:
import numpy as np
import pandas as pd
from utlis import train_test_split_data
import pandas
import evaluate
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

### Loading and preparing Data

In [2]:
df = pd.read_pickle("../data/combined_df.pkl")
df["content_text_joined"] = df["content_words"].apply(lambda x: " ".join(x))
TEXT_COL = "content_text_joined"    
LABEL_COL = "is_spoiler"      

# Split
X_train,_ , y_train, _ = train_test_split_data(
    df,
    TEXT_COL,
    LABEL_COL
)

In [3]:
ds={'text':X_train,'labels':y_train.astype(int)}
dataset = Dataset.from_dict(ds)

In [4]:
split_ds=dataset.train_test_split(test_size=0.2,seed=42)

### Training

In [5]:
model_checkpoint = "bert-base-uncased"
num_labels = 2 
BATCH_SIZE = 20
LEARNING_RATE = 3e-5
NUM_EPOCHS = 5
OUTPUT_DIR="./results/Bert"

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def tokenize(samples):
    return tokenizer(samples['text'], return_tensors="pt", padding=True,truncation=True)
tokenized_ds = split_ds.map(tokenize, batched=True)

Map:   0%|          | 0/56000 [00:00<?, ? examples/s]

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

In [8]:
for name, param in model.bert.named_parameters():
    param.requires_grad = False
for name, param in model.bert.pooler.named_parameters():
    param.requires_grad = True

In [9]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",      
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.555400,0.533251,0.733500
2,0.546300,0.528537,0.734500
3,0.541200,0.525773,0.738786
4,0.535800,0.523940,0.738643
5,0.534100,0.522445,0.739000


TrainOutput(global_step=14000, training_loss=0.5448524758475167, metrics={'train_runtime': 6170.8818, 'train_samples_per_second': 45.374, 'train_steps_per_second': 2.269, 'total_flos': 7.36710955008e+16, 'train_loss': 0.5448524758475167, 'epoch': 5.0})

In [19]:
trainer.save_model("../Models/Bert/final_model")
tokenizer.save_pretrained("../Models/Bert/tokenizer")

('../Models/Bert/tokenizer\\tokenizer_config.json',
 '../Models/Bert/tokenizer\\special_tokens_map.json',
 '../Models/Bert/tokenizer\\vocab.txt',
 '../Models/Bert/tokenizer\\added_tokens.json',
 '../Models/Bert/tokenizer\\tokenizer.json')